---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import re
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    f = open("university_towns.txt", "rb")
    data = []
    state = ""
    for line in f:
        string = line.decode()
        
        #do some clean work first:
        if 'The Colleges of Worcester Consortium:' in string:
            string = 'The Colleges of Worcester Consortium:'
        if 'The Five College Region of Western Massachusetts:' in string:
            string = 'The Five College Region of Western Massachusetts:'
        if 'North Mankato, South Central College' in string:
            string = 'North Mankato, South Central College'
        if 'Faribault, South Central College' in string:
            string = 'Faribault, South Central College'
        if 'Cecil B. Moore, Philadelphia, also known as "Templetown"' in string:
            string ='Cecil B. Moore, Philadelphia, also known as "Templetown"'
            
        if 'edit' in string:
            state = string
        else:
            data.append([state, string])

    df = pd.DataFrame(data, columns=["State", "RegionName"])

    #remove [edit] from State:
    df['State'].replace(to_replace="(.*)(\[.*\n)", value=r"\1", regex=True, inplace=True)

    #remove " (..." from RegionName:
    df['RegionName'].replace(to_replace="(\w*)( \(.*\n)", value=r"\1", regex=True, inplace=True)
    
    return df

df = get_list_of_university_towns()
df.head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', skiprows=[i for i in range(0,219)], parse_cols = 'E:G')
    df.rename(columns={'1999q4': 'quarter', 9926.1:'current', 12323.3:'2009'}, inplace=True)
    
    gdp = 0
    quarter = ''
    consecutive = 0
    for index, row in df.iterrows():
        if gdp == 0:
            gdp = row['2009']
            quarter = row['quarter']
            continue
        
        if row['2009'] < gdp:
            consecutive += 1
            if (consecutive == 1):
                gdp = row['2009']
                quarter = row['quarter']
                continue
            if (consecutive == 2):
                break
        else:
            gdp = row['2009']
            quarter = row['quarter']
            consecutive = 0
    
    return quarter

df = pd.read_excel('gdplev.xls', skiprows=[i for i in range(0,219)], parse_cols = 'E:G')
df.rename(columns={'1999q4': 'quarter', 9926.1:'current', 12323.3:'2009'}, inplace=True)

print(get_recession_start())
pd.set_option('display.max_rows', len(df))
print(df)
pd.reset_option('display.max_rows')

2008q3
   quarter  current     2009
0   2000q1  10031.0  12359.1
1   2000q2  10278.3  12592.5
2   2000q3  10357.4  12607.7
3   2000q4  10472.3  12679.3
4   2001q1  10508.1  12643.3
5   2001q2  10638.4  12710.3
6   2001q3  10639.5  12670.1
7   2001q4  10701.3  12705.3
8   2002q1  10834.4  12822.3
9   2002q2  10934.8  12893.0
10  2002q3  11037.1  12955.8
11  2002q4  11103.8  12964.0
12  2003q1  11230.1  13031.2
13  2003q2  11370.7  13152.1
14  2003q3  11625.1  13372.4
15  2003q4  11816.8  13528.7
16  2004q1  11988.4  13606.5
17  2004q2  12181.4  13706.2
18  2004q3  12367.7  13830.8
19  2004q4  12562.2  13950.4
20  2005q1  12813.7  14099.1
21  2005q2  12974.1  14172.7
22  2005q3  13205.4  14291.8
23  2005q4  13381.6  14373.4
24  2006q1  13648.9  14546.1
25  2006q2  13799.8  14589.6
26  2006q3  13908.5  14602.6
27  2006q4  14066.4  14716.9
28  2007q1  14233.2  14726.0
29  2007q2  14422.3  14838.7
30  2007q3  14569.7  14938.5
31  2007q4  14685.3  14991.8
32  2008q1  14668.4  14889.5
33  200

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=[i for i in range(0,219)], parse_cols = 'E:G')
    df.rename(columns={'1999q4': 'quarter', 9926.1:'current', 12323.3:'2009'}, inplace=True)
    
    start = get_recession_start()
    recessionBegin = 0
    consecutive = 0
    gdp = 0
    quarter = ''
    
    for index, row in df.iterrows():
        if recessionBegin == 0:
            if row['quarter'] != start:
                continue
            else:
                recessionBegin = 1
                gdp = row['2009']
        
        
        if row['2009'] > gdp:
            consecutive +=1
            if consecutive == 1:
                gdp = row['2009']
                continue
            if consecutive == 2:
                quarter = row['quarter']
                break;
        else:
            consecutive = 0;
            gdp = row['2009']
    
    return quarter
       
print(get_recession_end())

2009q4


In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=[i for i in range(0,219)], parse_cols = 'E:G')
    df.rename(columns={'1999q4': 'quarter', 9926.1:'current', 12323.3:'2009'}, inplace=True)
    
    start = get_recession_start()
    end = get_recession_end()
    lowestGDP = 0
    beginCount = 0
    bottom = ''

    for index, row in df.iterrows():
        if beginCount == 0:
            if row['quarter'] != start:
                continue;
            else:
                beginCount = 1;
                lowestGDP = row['2009']
                bottom = row['quarter']
        
        if row['2009'] < lowestGDP:
            lowestGDP = row['2009']
            bottom = row['quarter']
        
        if row['2009'] == end:
            break;
    
    return bottom

print(get_recession_bottom())

2009q2


In [31]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    qStrings = []
    for i in range(2000, 2017):
        qStrings.append(str(i)+'q1')
        qStrings.append(str(i)+'q2')
        qStrings.append(str(i)+'q3')
        qStrings.append(str(i)+'q4')
    qStrings.remove('2016q4')

    month = {1:'01', 2:'02', 3:'03', 4:'04', 5:'05', 6:'06', 7:'07', 8:'08', 9:'09', 10:'10', 11:'11', 12:'12'}
    
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    #remove useless columns:
    df.drop(df.columns[[0,3,4,5]], axis=1, inplace=True)
    df.drop(df.columns[[i for i in range(2,47)]], axis=1, inplace=True)
    
    #the table does have data for 2016-09, so fill it with NaN:
    df['2016-09'] = np.NaN
    
    for i in qStrings:
        #add a new column which will record the quarter mean GDP value:
        df[i] = 0
        
        (year, quarter) = re.split("q", i)
        quarter = int(quarter)
        
        first = year + '-' + month[quarter*3-2]
        second = year + '-' + month[quarter*3-1]
        third = year + '-' + month[quarter*3]
        
        if (i != '2016q3'):
            #use div(), not /, otherwise notebook crashes:
            df[i] = (df[first] + df[second] + df[third]).div(3)
        else:
            #2016q3 only has two month of data available:
            df[i] = (df[first] + df[second]).div(2)
        

    #remove additional columns:    
    df.drop(df.columns[[i for i in range(2,203)]], axis=1, inplace=True)
    
    #change State name from two letter to full name:
    df['State'] = df['State'].map(lambda x: states[x])
    
    df.set_index(["State","RegionName"], inplace=True)
        
    return df


df = convert_housing_data_to_quarters()
df.head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,NaN
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,NaN
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,NaN
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,NaN
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,NaN


In [30]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    dfUniversity = get_list_of_university_towns()
    dfUniversity.set_index(["State","RegionName"], inplace=True)
    ind = dfUniversity.index.values
    
    start = get_recession_start()
    
    #to find quarter before recession start:
    qStrings = []
    for i in range(2000, 2017):
        qStrings.append(str(i)+'q1')
        qStrings.append(str(i)+'q2')
        qStrings.append(str(i)+'q3')
        qStrings.append(str(i)+'q4')
    qStrings.remove('2016q4')
    
    bef_start = qStrings[qStrings.index(start) - 1]
    
    bottom = get_recession_bottom()
    
    dfHouse = convert_housing_data_to_quarters()
    dfHouse = dfHouse[[bef_start, bottom]]
    
    dfHouse['ratio'] = dfHouse[bef_start].div(dfHouse[bottom])
    
    dfUniTown = dfHouse.loc[ind]
    dfNonUniTown = dfHouse[~dfHouse.index.isin(ind)]
    
    dfUniTown.dropna(inplace=True)
    dfNonUniTown.dropna(inplace=True)
    
    (stat, p) = ttest_ind(dfUniTown['ratio'], dfNonUniTown['ratio'], equal_var=False)
    
    different = True if (p < 0.01) else False
    
    better = "university town" if (np.mean(dfUniTown['ratio']) < np.mean(dfNonUniTown['ratio'])) else "non-university town"
    
    return (different, p, better)

res = run_ttest()
print(res)
print('Type test:', 'Passed' if type(
    res) == tuple else 'Failed')
print('test `better` item type:','Passed' if type(res[0]) == bool or type(res[0]) == np.bool_ else 'Failed' )
print('`p` item type test:', 'Passed' if type(
    res[1]) == np.float64 else 'Failed')
print('`different` item type test:',
      'Passed' if type(res[2]) == str else 'Failed')
print('`different` item spelling test:', 'Passed' if res[2] in [
      "university town", "non-university town"] else 'Failed')

(True, 0.00041251360888714316, 'university town')
Type test: Passed
test `better` item type: Passed
`p` item type test: Passed
`different` item type test: Passed
`different` item spelling test: Passed


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
